# Migration Facility

<!-- SUMMARY: Migration of one or several variables from one data organization to another. Particular use of the Ball Tree sorting. -->

<!-- CATEGORY: Data_Base_Management -->

This case study is meant to demonstrate how to use *gstlearn* for migrating information from Db to DbGrid and vice-versa. Note that, in order to test the whole possibilities, we add a selection to both files.

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import matplotlib.pyplot as plt
import numpy as np

gdoc.setNoScroll()

Global parameters.

In [ ]:
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN,ndim)

Generate initial data set

In [ ]:
data = gl.Db.createFillRandom(ndat=20, ndim=ndim, nvar=1)
data.addSelectionByRanks(np.arange(2,18))
data.display()

In [ ]:
res = gp.plot(data, nameSize="z")

Create a grid, over the [0,1] x [0,1]

In [ ]:
grid = gl.DbGrid.create([50,50],dx=[0.02,0.02])
grid.addSelectionFromDbByConvexHull(data,0.05)
grid.display()

Migrate the information from **data** to **grid**. The migration is limited to te cell to which each sample belongs.

In [ ]:
err = gl.migrate(data, grid, "z", namconv=gl.NamingConvention("Migrate-NoFill",False))
gp.raster(grid)
gp.plot(data)
plt.show()

In this second attempt, we use the **flag_fill** option to fill the whole grid. However we compensate by specifying a maximum filling distance. Moreover we make this maximum distance anisotropic for ckecking.

In [ ]:
err = gl.migrate(data, grid, "z", flag_fill=True, dmax=[0.1,0.2], 
                 namconv=gl.NamingConvention("Migrate-Fill",False))
gp.raster(grid)
gp.plot(data)
plt.show()

## Ball Tree

Ball Tree is a special algorithm which makes search much more efficient. The principle is to build a Ball Tree based on a first data set. Then, using this tree, it is easy to consider a second data base and to search for the set of samples of the first data base which are the closest neighbors of each sample of the second data base.

To delonstrate this facility, we consider the case where we cannot benefit from any specific oragnization (grid for example) for any of the two data bases. Moreover (although this is a demnstration file, not a bench marking one), we consider a dense data set for better legibility

In [ ]:
ndat = 10000
db = gl.Db.createFillRandom(ndat)

gp.symbol(db)
gp.decoration(title="Location of samples from the First Data Base")

We build the corresponding BallTree structure

In [ ]:
ball = gl.Ball(db, None, None, 10, False, 1)

We now consider one target location and ask for the set of corresponding neighboring samples (say 10 closest samples). The target site is selected as the center of the square.

In [ ]:
neigh_size = 500
center = [0.5, 0.5]
knn1 = ball.queryOneAsVD(gl.VectorDouble(center), neigh_size)

The returned argument is a C objet (not mapped explicitly for Pythonn ... yet). However, some (statc) functions are available to retreive the contents of this objects. We use *getIndices* which returns the vector of ranks of samples constituting the neighborhood of the target site.

In [ ]:
target = db.clone()
target.deleteColumn("Selection")
iuid = target.addSelectionByRanks(knn1.getIndices(), "Selection")

As a consequence of the choice of the *Euclidean distance*, the set of samples neighboring the target (represented as a black square) are located within a circle centered on the target

In [ ]:
gp.symbol(db, s=1)
gp.symbol(target, c="yellow", flagCst=True, s=1)
gp.sample(center, marker='s')
gp.decoration(title="Euclidean distance")

We can produce a similar figure changing the distance, from Euclidean to Manhattan.

In [ ]:
ball = gl.Ball(db, None, None, 10, False, 2)

knn2 = ball.queryOneAsVD(gl.VectorDouble(center), neigh_size)

target = db.clone()
target.deleteColumn("Selection")
iuid = target.addSelectionByRanks(knn2.getIndices(), "Selection")

As a consequence of the choice of the *Manhattan distance*, the set of samples neighboring the target (represented as a black square) are located within a diamond centered on the target

In [ ]:
gp.symbol(db, s=1)
gp.symbol(target, c="yellow", flagCst=True, s=1)
gp.sample(center, marker='s')
gp.decoration(title="Manhattan distance")